In [10]:
import IPython.display as ipd
import numpy as np

from src.services import get_wave_provider

In [15]:
bads = ["XC579430.ogg", "XC590621.ogg"]

In [16]:
w = get_wave_provider()

In [20]:
x = w.get_audio_fragment(bads[1])

In [21]:
np.where(x != 0)

(array([], dtype=int64),)